In [1]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, SKOS, SDO, XSD

In [2]:
g = Graph()
g.parse("movie.ttl")

f_data = "http://projet.fr/films_data/"
ns_film_data = Namespace(f_data)

f_schema = "http://projet.fr/films_schema/"
ns_film_schema = Namespace(f_schema)

g.bind("", ns_film_data)
g.bind('f_schema', ns_film_schema)

for property_to_split in [ns_film_schema.stars, ns_film_schema.genre]:
    for subj, pred, obj in g.triples((None, property_to_split, None)):
        print(pred)
        values = obj.split(",")  # Split the string by ',' delimiter

        # Remove existing triple with the property to split
        g.remove((subj, pred, obj))

        # Create new triples with individual values
        for value in values:
            # Create a new property for each value
            new_property = property_to_split  # Replace with your new property URI
            g.add((subj, new_property, Literal(value.strip())))  # Add new triple

http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schema/stars
http://projet.fr/films_schem

In [3]:


p_data = "http://projet.fr/perso_data/"
ns_perso_data = Namespace(p_data)

p_schema = "http://projet.fr/perso_schema/"
ns_perso_schema = Namespace(p_schema)

g.bind("p_schema", ns_perso_schema)


def cleanHead(head):
    return ''.join(letter for letter in head if letter.isalnum())


for subj, property, value in g:
        
    if property.split('/')[-1] == 'director':
        head = bytes(value, 'iso-8859-1').decode('utf-8')
        head = cleanHead(head)
        head = URIRef(f"{f_data}{head}")
        
        g.add((head, RDF.type, SDO.Person))
        g.add((head, RDF.type, ns_perso_schema.Real))
        g.add((head, SDO.name, value))
        
        g.remove((subj, property, value))
        g.add((subj, SDO.director, head))
        
    if property.split('/')[-1] == 'name':
        g.remove((subj, property, value))
        g.add((subj, SDO.name, value))
    
    if property == ns_film_schema.stars:
        head = bytes(value, 'iso-8859-1').decode('utf-8')
        head = cleanHead(head)
        head = URIRef(f"{f_data}{head}")

        g.add((head, RDF.type, SDO.Person))
        g.add((head, RDF.type, ns_perso_schema.Real))
        g.add((head, SDO.name, value))

        g.remove((subj, property, value))
        g.add((subj, property, head))

    if property == ns_film_schema.entity:
        head = cleanHead(value)
        head = URIRef(f"{f_schema}{head}")

        g.add((head, RDF.type, SKOS.Concept))
        g.add((head, SKOS.prefLabel, value))
        g.add((head, SKOS.broader, ns_film_schema["producerConcept"]))

        g.remove((subj, property, value))
        g.add((subj, property, head))

    if property == ns_film_schema.genre:
        head = cleanHead(value)
        head = URIRef(f"{f_schema}{head}")

        g.add((head, RDF.type, SKOS.Concept))
        g.add((head, SKOS.prefLabel, value))  # Add new triple
        g.add((head, SKOS.broader, ns_film_schema["genreConcept"]))

        g.remove((subj, property, value))
        g.add((subj, property, head))

    if property == ns_film_schema.year:
        g.remove((subj, property, value))
        g.add((subj, property, Literal(value, datatype=XSD.gYear)))

    if property == RDF.type and value.split('/')[-1] == 'movie':
        g.remove((subj, property, value))
        g.add((subj, property, SDO.Movie))

In [4]:
g.serialize(destination="modified_movie.ttl", format="turtle")

<Graph identifier=N375d54b9a90c4a6fb0866f583f0e6c57 (<class 'rdflib.graph.Graph'>)>